## <center>HMMs from smaller epochs of data</center>

##### MUST SHARE KERNEL WITH NOTEBOOK `00-load-data.ipynb`

#### Calculate HMMs from growing portions of long trajectory
------------

--------------------
 
INCOMPLETE TODO FIXME the second loop is basically hardcoded for kmeans

In [38]:
total = len(good_ones)*(len(epochsizes)+1)
howmany = 0

for feat, nm, dataset in filter(
    lambda x: (
        x[1] in set(list(zip(*good_ones))[0])),
    aswa_tools.iter_models(all_models)
):

    if "hmm" not in dataset:
        dataset["hmm"] = dict()

    for (_nm, clust, _n, _tica_lag,
        _clustpar, _hmm_lag) in filter(
        lambda x: (x[0] == nm),
        _good_ones
    ):

        if clust not in dataset["hmm"]:
            dataset["hmm"][clust] = list()

        hmmsetups = dataset["hmm"][clust]
        
        dds  = da / _nm
        dhmm = dds/"hmm"/(
            "tica_lag_{}".format(_tica_lag[1]))
        fhmm = dhmm/"results.pyemma"

        if not dhmm.is_dir():
            dhmm.mkdir()

        if set(_clustpar) != {None}:
            
            print(colorama.Back.BLUE)
            print(colorama.Fore.BLACK)
            print("Retrieving clustering model")
            
            clustsetup = correctone(
                dataset["cluster_%s"%clust],
                *_clustpar)
            
            print("Got this one:")
    #        print(colorama.Back.RESET)
     #       print(colorama.Fore.BLUE)

        else:
            
            print(colorama.Back.YELLOW)
            print(colorama.Fore.BLACK)
            print((
                "GETTING from 2 state model",
                ", no upstream clustering"))
            
            clustsetup = dict(
                byepochsize = {
                    _tica_lag : [
    (es, [the_dtrajs[bestone][:es]])
    for es in epochsizes+[the_dtrajs[bestone].shape[0]]
            ]})
            
            pprint(clustsetup)

        if "byepochsize" not in clustsetup:
            print(colorama.Fore.BLACK)
            print(colorama.Back.YELLOW)
            print("no BES clustering found")
            print(_nm, clust, _n, _tica_lag,
                _clustpar, _hmm_lag)
            
            continue
        
        # Other by epoch size calculatations
        # didn't create their own setup. 
        # Upstream HMMs were made inside the
        # ITS objects, recreating setup for
        # simplicity here
        reversible=False
        setup = {
            "kwargs": dict(
                nstates=_n[1],
                lag=_hmm_lag[1],
                reversible=reversible,),
            "result": False,
            "inpar": (_tica_lag, _clustpar),
            "byepochsize": list(),
        }

        hmmsetups.append(setup)
        hmm_bes = setup["byepochsize"]

        thisone = "%s-%s_%d-%s_%d-%s_%s-%s"%(
            clust, "nstates", _n[1],
            "hmmlag", _hmm_lag[1],
            "reversible", str(reversible),
            _model_name(feat,
                clustsetup.get("kwargs",
                    dict(interfaces="2state"))))
        
        fbes = fhmm.with_suffix(
            ".%s"%thisone)
        
        print(colorama.Fore.BLACK)
        print(colorama.Back.MAGENTA)
        print("thisone: %s"%thisone)
#        print(colorama.Back.BLUE)
#        print("clustsetup:\n{}".format(
#            pformat(clustsetup)))

        for __tica_lag in filter(
            lambda x: x==_tica_lag,
            clustsetup["byepochsize"]
        ):
            
            __, tica_lag = __tica_lag
            
            for es,clustbes in clustsetup[
                "byepochsize"][_tica_lag
            ]:
                
                if not clustbes:
                    
                    print(colorama.Back.YELLOW)
                    print(colorama.Fore.BLACK)
                    print((
        "Upstream calculations were problematic",
        " for this one"+pformat(("epochsize: %d" %
            es, _nm, clust, _n, _tica_lag,
            _clustpar, _hmm_lag))))
                    
                    continue
                
                howmany += 1
                
                print(colorama.Fore.BLACK)
                print(colorama.Back.CYAN)
                print("Doing the HMM calcs")
                print(colorama.Back.RESET)
                print(colorama.Fore.GREEN)
                print("On to number %d/%d"%(
                    howmany, total))
                
                _fbes = Path(
                    str(fbes) + \
                    "-byepochsize-%dns" % es)

                print("created the path")
                print(_fbes)

                if es > 0:
                    if _fbes.is_file():
                        print("found byepochsize file")

                        result = pyemma.load(_fbes)

                    elif clust == "2-state":

                        clustinp = clustbes
                        
                        # Already 2-state, invalid
                        # argument for regular MSM
                        if "nstates" in setup["kwargs"]:
                            del setup["kwargs"]["nstates"]
                        
                        try:
                            result = \
    pyemma.msm.bayesian_markov_model(
        clustinp,
        # These kwargs dont' really help
        # with the file-name-based model
        # identification scheme
        # (and/or have white space)
        dt_traj="0.02 ns",
        nsamples=20,
        **setup["kwargs"],)

                            result.save(_fbes)
                            
                        except:

                            result = False
                            
                            print(colorama.Back.YELLOW)
                            print(
        "2-State MSM calculation failed")
                            print(pformat(
        _nm, clust, _n, _tica_lag, _clustpar, _hmm_lag))
                                            
                    else:
                        
                        clustinp = clustbes.dtrajs

                        print(colorama.Back.BLUE)
                        print(colorama.Fore.BLACK)
                        print(pformat(len(clustinp)))
                        print([dt.shape for dt in clustinp])
                        
                        try:

                            result = \
    pyemma.msm.bayesian_hidden_markov_model(
        clustinp,
        # These kwargs dont' really help
        # with the file-name-based model
        # identification scheme
        # (and/or have white space)
        dt_traj="0.02 ns",
        nsamples=20,
        **setup["kwargs"],)
                            
                            result.save(_fbes)
                            
                        except:
                            
                            result = False
                            
                            print(colorama.Back.YELLOW)
                            print("HMM calculation failed")
                            print(pformat([
        _nm, clust, _n, _tica_lag, _clustpar, _hmm_lag]))
                                            
#                elif es < 0:
 #                   print("Retrieving full data model")
#
 #                   result = aswa_tools.get_matching_input(
  #                      all_models[feat_Ca]["ltj"]["hits"]["kmeans"],
   #                     *_n
    #                ).models[tica_lags.index(tica_lag)]
     #               
      #              es = result.dtrajs_full[0].shape[0]

                else:
                    print(colorama.Back.YELLOW)
                    print("How can have zero epochsize?")
                    
                    continue
                    
                hmm_bes.append((
                    # Actual amount of data
                    es * (dataset["datashape"][0][0] - 1)
                    if dataset["n_trajs"]>1
                    else es,
                    # Rate result
                    result))

  # TODO actual data length should be calculated by
  #      factoring each datashape element into the
  #      the calculation instead of assuming the
  #      block structure and only first traj length
  #      [
   #         es * (dataset["datashape"][0][0] - 1)
    #        for es in aswa_tools.workloadbounds(
     #       dataset)[1:]
      #  ]
    
    
print(colorama.Back.LIGHTBLACK_EX + \
      colorama.Fore.BLUE + \
      "  DONE LOADING DATA  ")



('GETTING from 2 state model', ', no upstream clustering')
{'byepochsize': {(None, None): [(25000,
                                 [array([1, 1, 1, ..., 0, 0, 0], dtype=int32)]),
                                (50000,
                                 [array([1, 1, 1, ..., 0, 0, 0], dtype=int32)]),
                                (100000,
                                 [array([1, 1, 1, ..., 0, 0, 0], dtype=int32)]),
                                (200000,
                                 [array([1, 1, 1, ..., 0, 0, 0], dtype=int32)]),
                                (400000,
                                 [array([1, 1, 1, ..., 1, 1, 1], dtype=int32)]),
                                (600000,
                                 [array([1, 1, 1, ..., 0, 0, 0], dtype=int32)]),
                                (800000,
                                 [array([1, 1, 1, ..., 0, 0, 0], dtype=int32)]),
                                (1000000,
                                 [array([1, 1

----
Older version of above loop with different middle iteration setups

In [39]:
"""
# Figure out a flat way to iterate for gosh sakes
#for nm, clust, _n, _lag, _k in good_ones:
#    ticasetup = correctone(
 #       all_models[feat_Ca][nm]["tica"],
  #      key="lag", val=_lag[1])

total = len(_good_ones)*(len(epochsizes)+1)
howmany = 0

for feat, nm, dataset in filter(
    lambda x: (
        x[1] in set(list(zip(*good_ones))[0])),
    aswa_tools.iter_models(all_models)
):
#    # should get all the non-byepochsize guys
 #   # and roll to the clust calculation
  #  # FIXME THIS needs to use the `set` of key-val
   # # pairs that constitute all the good ones
    ## clust_pars, then cat the clust method lists
#    # together for iteration and filter by the
 #   # entries from that `set`
  #  for clustsetup in filter(
   #     lambda x: (
    #        "byepochsize" in x) and (
     #       x["kwargs"]["lag"] in _good_clust_lags),
     #   dataset["kmeans"]
    #):
   #     clust_par = clustsetup["kwargs"]["k"]
    for ticasetup in filter(
        lambda x: (
            "byepochsize" in x) and (
            x["kwargs"]["lag"] in _good_tica_lags),
        dataset["tica"]
    ):
        tica_lag = ticasetup["kwargs"]["lag"]
        
        for _nm, clust, _n, _tica_lag, _clustpar, _hmm_lag in filter(
        #for goodone in filter(
            lambda x: (x[0] == nm
                ) and (x[3][1] == tica_lag),
            _good_ones
        ):
            dds  = da / nm
            dhmm = dds/"hmm"/("tica_lag_%d"%_tica_lag[1])
            fhmm = dhmm/"results.pyemma"
            
            if not dhmm.is_dir():
                dhmm.mkdir()

            hmmsetup = 
            thisone = _model_name(feat, clustsetup["kwargs"])
            fbes = fhmm.with_suffix(".%s"%thisone)
            
            if "byepochsize" not in clustsetup:
                clustsetup["byepochsize"] = list()
                
            hmm_bes = hmmsetup["byepochsize"]
            
            for es,clustbes in clustsetup["byepochsize"]:

                
                dits["kwargs"] = dict(
                    lags=tica_lags,
                    nstates=n,
                    reversible=reversible,
                    #nits=3,
                    errors="bayes",
                    nsamples=20,
                )
                
                
                
                
                
                if "hmm" not in all_models[feat][nm]:
                    all_models[feat][nm]["hmm"] = dict()
                    all_models[feat][nm]["hmm"][clust] = list()
                
                setups = all_models[feat][nm]["hmm"][clust]
                
                setup = {
                    "par"        : _k,
                    "byepochsize": list()
                    "kwargs"     : {k:v for k,v in [
                        _n,
                        _hmm_lag,
                        ("reversible", False),
                        ("nsamples",   20),
                        #("errors",     "bayes"),
                    ]},
                }
            
            try:
                setup["result"] = \
                pyemma.msm.bayesian_hidden_markov_model(
                    clust_bes[-1][1].dtrajs,
                    **setup["kwargs"],)
            except:
                setup["result"] = False
                
            setups.append((es, setup))
"""

'\n# Figure out a flat way to iterate for gosh sakes\n#for nm, clust, _n, _lag, _k in good_ones:\n#    ticasetup = correctone(\n #       all_models[feat_Ca][nm]["tica"],\n  #      key="lag", val=_lag[1])\n\ntotal = len(_good_ones)*(len(epochsizes)+1)\nhowmany = 0\n\nfor feat, nm, dataset in filter(\n    lambda x: (\n        x[1] in set(list(zip(*good_ones))[0])),\n    aswa_tools.iter_models(all_models)\n):\n#    # should get all the non-byepochsize guys\n #   # and roll to the clust calculation\n  #  # FIXME THIS needs to use the `set` of key-val\n   # # pairs that constitute all the good ones\n    ## clust_pars, then cat the clust method lists\n#    # together for iteration and filter by the\n #   # entries from that `set`\n  #  for clustsetup in filter(\n   #     lambda x: (\n    #        "byepochsize" in x) and (\n     #       x["kwargs"]["lag"] in _good_clust_lags),\n     #   dataset["kmeans"]\n    #):\n   #     clust_par = clustsetup["kwargs"]["k"]\n    for ticasetup in filter(\n 

-------------

#### $\infty$ DONE
-------

In [40]:
print(colorama.Back.LIGHTBLACK_EX + \
      colorama.Fore.BLUE + \
      "  DONE LOADING DATA  ")

  DONE LOADING DATA  


--------
--------

-----------

-----------

-----------

-----------

-----------

-----------

-----------

-----------

-----------